In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from datetime import time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, time, date
from scipy.stats import t
from collections import defaultdict
import matplotlib

# Data prepration
We need a dataset that has start_time, end_time, V1, V2


In [47]:
df = pd.read_csv(r"Real_Data\primaryschool.csv", sep="\t", header=None)

merged = defaultdict(set)

for node_col, attr_col in [(1, 3), (2, 4)]:
    for node, attrs in df.groupby(node_col)[attr_col]:
        merged[node].update(attrs)

meta_df = (
    pd.DataFrame(
        [(node, next(iter(attrs))) for node, attrs in merged.items()],
        columns=["node", "attribute"],
    )
    .sort_values("attribute")
    .reset_index(drop=True)
)

node_mapping = dict(zip(meta_df["node"], meta_df.index))
node_to_attribute = dict(zip(meta_df.index, meta_df["attribute"]))

df = df[[0, 1, 2]]
df.columns = ["starting_times", "source_nodes", "target_nodes"]

df["source_nodes"] = df["source_nodes"].map(node_mapping)
df["target_nodes"] = df["target_nodes"].map(node_mapping)


MIN_TIME = 8.5 * 3600
df["starting_times"] = df["starting_times"] - MIN_TIME
df = df[df["starting_times"] > 24 * 3600].reset_index(drop=True)
df["starting_times"] = df["starting_times"] - 24 * 3600
df['starting_times']=df['starting_times']-df['starting_times'].min()
df["ending_times"] = df["starting_times"] + 20
df['durations']=df["ending_times"] -df["starting_times"] 

In [50]:
df.to_csv('Pre_processed_Data\primary_school_day2.csv', index=False)

In [41]:
from flowstab.flow_stability import FlowStability
fs = FlowStability()
fs.state.next

2025-11-11 10:38:22,499 - INFO - flowstab\flow_stability.py:235 - PID:56436 - Set an empty temporal network.
2025-11-11 10:38:22,500 - INFO - flowstab\flow_stability.py:139 - PID:56436 - Successfully initiated a FlowStability instance!


([], 'set_temporal_network')

In [ ]:

# check what to do next
fs.state.next
# Out[3]: ([], 'set_temporal_network')
# > you need to run `set_temporal_network` - how should this be done:
fs.state.howto['set_temporal_network']
# print(fs.state.howto['set_temporal_network'])
# 
#         Set the temporal network for the flow stability analysis.
# 
#         Parameters
#         ----------
#         **kwargs : dict
#             Arguments to initialize a ContTempNetwork instance.
#         ...
fs.set_temporal_network(events_table="my_contacts.csv")
fs_mice.state.next
# Out[6]: ([], 'compute_laplacian_matrices')
fs.compute_laplacian_matrices()
fs_mice.state.next
# Out[8]: (['time_scale'], 'compute_inter_transition_matrices')
# So we need to set `time_scale` next, but how?
print(fs_mice.state.howto['time_scale'])
# Set the time scale(s) for the random walk's transition rate.
# 
# .. note::
#     You might also use `set_time_scale` to directly create a range of
#     time scales.
# 
# Parameters
# ----------
# time_scale : None, int, float, or iterator of float
#     If None, a default value is used.
#     If an int or float, a single time scale is set.
#     If an iterator, it must yield float or int values.
# 
# ...
fs.set_time_scale(10)
fs_mice.state.next
# Out[11]: ([], 'compute_inter_transition_matrices')
fs.compute_inter_transition_matrices()
fs.time_direction = 0  # perform both forward and backard
fs.set_flow_clustering()
fs.find_louvain_clustering()
fs.flow_clusetering_backward
# Out[17]: {10: <flowstab.network_clustering.FlowIntegralClustering at 0x7fef57a7fd00>}